In [1]:
# Creating a open ai assistant

import os
from autogen import config_list_from_json
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent

assistant_id = os.environ.get("ASSISTANT_ID", None)
config_list = config_list_from_json("OAI_CONFIG_LIST")
llm_config = {
    "config_list": config_list,
}
assistant_config = {
    # define the openai assistant behavior as you need
}
oai_agent = GPTAssistantAgent(
    name="oai_agent",
    instructions="I'm an openai assistant running in autogen",
    llm_config=llm_config,
    assistant_config=assistant_config,
)

c:\Users\sides\.conda\envs\autogen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
OpenAI client config of GPTAssistantAgent(oai_agent) - model: gpt-4
Matching assistant found, using the first matching assistant: {'id': 'asst_SiTity8cWf4EDjP0eJc3gM45', 'created_at': 1733821303, 'description': None, 'instructions': "I'm an openai assistant running in autogen", 'metadata': {}, 'model': 'gpt-4', 'name': 'oai_agent', 'object': 'assistant', 'tools': [], 'response_format': 'auto', 'temperature': 1.0, 'tool_resources': ToolResources(code_interpreter=None, file_search=None), 'top_p': 1.0}


# Code Interpreter
The Code Interpreter empowers your agents to write and execute Python code in a secure environment provide by OpenAI. This unlocks several capabilities, including but not limited to:

Process data: Handle various data formats and manipulate data on the fly.
Generate outputs: Create new data files or even visualizations like graphs.

### Using openai platform

In [ ]:
assistant_config = {
    "tools": [
        {"type": "code_interpreter"},
    ],
    "tool_resources": {
        "code_interpreter": {
            "file_ids": ["$file.id"]  # optional. Files that are passed at the Assistant level are accessible by all Runs with this Assistant.
        }
    }
}

### Using code

In [ ]:
from openai import OpenAI
client = OpenAI(
    # Defaults to os.environ.get("OPENAI_API_KEY")
)
# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("mydata.csv", "rb"),
  purpose='assistants'
)

## File Search
The File Search tool empowers your agents to tap into knowledge beyond its pre-trained model. This allows you to incorporate your own documents and data, such as product information or code files, into your agent's capabilities.

### Using openai platform

In [ ]:
assistant_config = {
    "tools": [
        {"type": "file_search"},
    ],
    "tool_resources": {
        "file_search": {
            "vector_store_ids": ["$vector_store.id"]
        }
    }
}

### Using code

In [ ]:
from openai import OpenAI
client = OpenAI(
    # Defaults to os.environ.get("OPENAI_API_KEY")
)

# Step 1: Create a Vector Store
vector_store = client.beta.vector_stores.create(name="Financial Statements")
print("Vector Store created:", vector_store.id)  # This is your vector_store.id

# Step 2: Prepare Files for Upload
file_paths = ["edgar/goog-10k.pdf", "edgar/brka-10k.txt"]
file_streams = [open(path, "rb") for path in file_paths]

# Step 3: Upload Files and Add to Vector Store (with status polling)
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Step 4: Verify Completion (Optional)
print("File batch status:", file_batch.status)
print("Uploaded file count:", file_batch.file_counts.processed)

## Function calling
Function Calling empowers you to extend the capabilities of your agents with your pre-defined functionalities, which allows you to describe custom functions to the Assistant, enabling intelligent function selection and argument generation.

In [ ]:
# learn more from https://platform.openai.com/docs/guides/function-calling/function-calling
from autogen.function_utils import get_function_schema

def get_current_weather(location: str) -> dict:
    """
    Retrieves the current weather for a specified location.

    Args:
    location (str): The location to get the weather for.

    Returns:
    Union[str, dict]: A dictionary with weather details..
    """

    # Simulated response
    return {
        "location": location,
        "temperature": 22.5,
        "description": "Partly cloudy"
    }

api_schema = get_function_schema(
    get_current_weather,
    name=get_current_weather.__name__,
    description="Returns the current weather data for a specified location."
)

assistant_config = {
    "tools": [
        {
            "type": "function",
            "function": api_schema,
        }
    ],
}